## **Dataset**

In [1]:
from datasets import load_dataset

ds = load_dataset("datablations/c4-filter-small", split="train")
ds = ds.select_columns(["text"])
ds = ds.train_test_split(test_size=0.1)

In [2]:
ds


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 10000
    })
})

## **Tokenizer**

In [3]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.normalizers import NFKC
from tokenizers.decoders import ByteLevel as ByteLevelDecoder

# Initialize BPE tokenizer
tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = ByteLevel()
tokenizer.normalizer = NFKC()
tokenizer.decoder = ByteLevelDecoder()

trainer = BpeTrainer(
    vocab_size=100_000,
    special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"]
)

tokenizer.train_from_iterator(ds["train"]["text"], trainer)
tokenizer.save("gpt_tokenizer.json")

In [4]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(tokenizer_file="gpt_tokenizer.json")
tokenizer.add_special_tokens({
    "bos_token": "<s>",
    "eos_token": "</s>",
    "unk_token": "<unk>",
    "pad_token": "<pad>",
    "mask_token": "<mask>",
})

tokenizer.save_pretrained("gpt-tokenizer")

c:\Users\Admin\miniconda3\envs\tf-gpu\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


('gpt-tokenizer\\tokenizer_config.json',
 'gpt-tokenizer\\special_tokens_map.json',
 'gpt-tokenizer\\tokenizer.json')

In [5]:
len(tokenizer)


100000

In [6]:
tokenizer.pad_token_id, tokenizer.eos_token_id, tokenizer.bos_token_id


(1, 2, 0)

In [7]:
def tokenize(example):
    return tokenizer(example["text"])

tokenized_ds = ds.map(
    tokenize, remove_columns=["text"], batched=True
)


Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [8]:
tokenized_ds


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10000
    })
})

In [9]:
block_size = 512

def group_texts(examples):
    # concat input_ids
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated["input_ids"])
    total_length = (total_length // block_size) * block_size

    # split block_size
    result = {
        k: [concatenated[k][i : i + block_size] for i in range(0, total_length, block_size)]
        for k in concatenated
    }

    # prepare labels
    result["labels"] = result["input_ids"].copy()
    return result

lm_ds = tokenized_ds.map(group_texts, batched=True)


Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [10]:
lm_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 77014
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 9032
    })
})

In [11]:
import torch

torch.tensor(lm_ds["train"]["input_ids"][:5])

tensor([[  257,   365, 30640,  ...,  1174,  3747,   237],
        [  743, 12350,   239,  ...,   233,  4343,   581],
        [  333,  3153,   266,  ...,   213,  1166,   240],
        [  213,  2050,   308,  ...,  3307,   240,   213],
        [16820,   272,   213,  ...,   240,  2852,  5791]])

In [12]:
lm_ds


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 77014
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 9032
    })
})

## **Model**

In [52]:
from transformers import GPT2Config, GPT2LMHeadModel

config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    n_positions=block_size,
    n_embd=512,
    n_layer=6,
    n_head=8,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

model = GPT2LMHeadModel(config)

In [14]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

training_args = TrainingArguments(
    output_dir="gpt-small-c4",
    logging_dir="logs",
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=20,
    eval_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",
    eval_steps=1000,
    save_steps=1000,
    logging_steps=1000,
    save_total_limit=1,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end=True,
    fp16=True,
    gradient_checkpointing=True,
    gradient_accumulation_steps=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_ds["train"],
    eval_dataset=lm_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

c:\Users\Admin\miniconda3\envs\tf-gpu\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


## **Training**

In [15]:
trainer.train()

  0%|          | 0/64180 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
c:\Users\Admin\miniconda3\envs\tf-gpu\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\Admin\miniconda3\envs\tf-gpu\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
c:\Users\Admin\miniconda3\envs\tf-gpu\lib\si

{'loss': 7.1748, 'grad_norm': 1.340687870979309, 'learning_rate': 4.92209411031474e-05, 'epoch': 0.31}


  0%|          | 0/753 [00:00<?, ?it/s]

{'eval_loss': 6.556571960449219, 'eval_runtime': 49.7847, 'eval_samples_per_second': 181.421, 'eval_steps_per_second': 15.125, 'epoch': 0.31}


...

  0%|          | 0/753 [00:00<?, ?it/s]

{'eval_loss': 4.535489559173584, 'eval_runtime': 37.0434, 'eval_samples_per_second': 243.822, 'eval_steps_per_second': 20.327, 'epoch': 19.94}


c:\Users\Admin\miniconda3\envs\tf-gpu\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\Admin\miniconda3\envs\tf-gpu\lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


{'train_runtime': 20930.0957, 'train_samples_per_second': 73.592, 'train_steps_per_second': 3.066, 'train_loss': 4.774919954361147, 'epoch': 20.0}


TrainOutput(global_step=64180, training_loss=4.774919954361147, metrics={'train_runtime': 20930.0957, 'train_samples_per_second': 73.592, 'train_steps_per_second': 3.066, 'total_flos': 8.950241713717248e+16, 'train_loss': 4.774919954361147, 'epoch': 20.0})

In [16]:
torch.cuda.empty_cache()

In [17]:
# push to hub
trainer.push_to_hub("binhphap5/gpt-small-c4")

model.safetensors:   0%|          | 0.00/282M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/binhphap5/gpt-small-c4/commit/e8d5bec2572a149fe86eb9a6fe8e3d205a0eec3a', commit_message='binhphap5/gpt-small-c4', commit_description='', oid='e8d5bec2572a149fe86eb9a6fe8e3d205a0eec3a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/binhphap5/gpt-small-c4', endpoint='https://huggingface.co', repo_type='model', repo_id='binhphap5/gpt-small-c4'), pr_revision=None, pr_num=None)

## **Inference**

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "binhphap5/gpt-small-c4"

model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [74]:
prompt = "We gotta get at least ten thousand"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

In [75]:
output = model.generate(
    **inputs,
    max_new_tokens=50, # This is just for testing purpose, can be any value but must not exceed (512 - len(prompt))
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.9,
)

In [76]:
print(tokenizer.decode(output[0], skip_special_tokens=True))

 We gotta get at least ten thousand dollars to get a book. The Book of Mormon has had more of a new book, and more of it was available on a library of books for readers. And it was a book read. A book I read recently. Most readers had been reading


In [97]:
import math
import torch
# Shift for labels (causal LM setting: predict token t+1 from token t)
labels = output[:, 1:].clone()
inputs = output[:, :-1].clone()

with torch.no_grad():
    outputs = model(inputs)
    logits = outputs.logits

# Compute log softmax over vocabulary
log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
# Gather log-probabilities corresponding to the labels
selected_log_probs = log_probs.gather(2, labels.unsqueeze(-1)).squeeze(-1)

# Sum negative log probs → total NLL
nll = -selected_log_probs.sum().item()
num_tokens = labels.numel()
perplexity = math.exp(nll / num_tokens)
perplexity

21.237168770906834